In [2]:
import pandas

import pygsheets
import numpy
import scipy

# import data directly from google sheets

In [6]:
gc = pygsheets.authorize(service_account_env_var='GDRIVE_API_CREDENTIALS')
spreadsheet = gc.open_by_key('1foPLE6K-uqFlaYgLPAUxzeXfDO5wOOqE7tibNHeqTek') # present-day version

#spreadsheet[1] "Gas Pipelines" tab is the second index
gas_pipes = spreadsheet.worksheet('title','Gas pipelines').get_as_df(start='A2')
oil_pipes = spreadsheet.worksheet('title', 'Oil/NGL pipelines').get_as_df(start='A2')

#pipes_df_orig = pandas.concat([oil_pipes, gas_pipes], ignore_index=True)
pipes_df_orig = oil_pipes.copy()

#get country ratios sheet
country_ratios_df = spreadsheet.worksheet('title', 'Country ratios by pipeline').get_as_df()

In [12]:
region_dict_df = spreadsheet.worksheet('title', 'Region dictionary').get_as_df(start='A2')

## replace "--" with NaN, removing empty rows

the dataset is structured to have -- wherever there's a lookup value that doesn't exist; replacing it with NaN (numpy.nan) allows pandas to treat it as a null value, which makes calculations much easier

In [13]:
# replace -- entries with NaN
pipes_df_orig = pipes_df_orig.replace('--', numpy.nan)
pipes_df_orig = pipes_df_orig[pipes_df_orig['PipelineName']!='']
pipes_df_orig = pipes_df_orig[pipes_df_orig['Wiki']!='']
country_ratios_df.replace('--', numpy.nan, inplace=True)

In [21]:
agt_countries = region_dict_df.loc[region_dict_df.AfricaGasTracker=='Yes']['Country'].tolist()

In [24]:
pipes_df_orig.loc[(pipes_df_orig.Fuel=='Oil')&
                  ~(pipes_df_orig.Countries.apply(
                      lambda x: set(x.split(', ')).isdisjoint(set(agt_countries))
                  ))].to_excel('Africa-Gas-Tracker-oil-pipelines.xlsx')